# Batch Gradient Descent ( Generally Not Preffered )

Memory Inefficient

No Better Convergence



# Simple Example

In [ ]:
from sklearn.datasets import make_classification
import torch

In [ ]:
# Step 1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [ ]:
print(X)
print("-----"*50)
print(y)

[[ 1.06833894 -0.97007347]
 [-1.14021544 -0.83879234]
 [-2.8953973   1.97686236]
 [-0.72063436 -0.96059253]
 [-1.96287438 -0.99225135]
 [-0.9382051  -0.54304815]
 [ 1.72725924 -1.18582677]
 [ 1.77736657  1.51157598]
 [ 1.89969252  0.83444483]
 [-0.58723065 -1.97171753]]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[1 0 0 0 0 1 1 1 1 0]


In [ ]:
X =  torch.tensor(X, dtype=torch.float32)
y =  torch.tensor(y, dtype=torch.long)

<ipython-input-5-a2dc24915e08>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X =  torch.tensor(X, dtype=torch.float32)
<ipython-input-5-a2dc24915e08>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y =  torch.tensor(y, dtype=torch.long)


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return self.features.shape[0]

  def __getitem__(self, index):

    return self.features[index], self.labels[index]

In [ ]:
dataset = CustomDataset(X, y)

In [ ]:
print("Length of Dataset: ", len(dataset))
print("----"*50)


Length of Dataset:  10
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [ ]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
for batch in dataloader:
  print(batch)
  print("----"*50)

[tensor([[-1.9629, -0.9923],
        [-0.5872, -1.9717]]), tensor([0, 0])]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[tensor([[-1.1402, -0.8388],
        [-0.9382, -0.5430]]), tensor([0, 1])]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[tensor([[-0.7206, -0.9606],
        [ 1.7774,  1.5116]]), tensor([0, 1])]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[tensor([[-2.8954,  1.9769],
        [ 1.0683, -0.9701]]), tensor([0, 1])]
-------------------------------------------------------------------------------------------------

# Cancer Dataset

In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
df =  pd.read_csv("/content/drive/MyDrive/PYTORCH/data.csv")

In [4]:
df.shape

(569, 33)

In [5]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

# Train Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Numpy arrays to Pytorch Tensors


In [9]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [10]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [11]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Defining The Model

In [14]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [15]:
learning_rate = 0.1
epochs = 25

In [16]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [17]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.12410373985767365
Epoch: 2, Loss: 0.14338769018650055
Epoch: 3, Loss: 0.06804819405078888
Epoch: 4, Loss: 0.192193865776062
Epoch: 5, Loss: 0.14085493981838226
Epoch: 6, Loss: 0.2810288965702057
Epoch: 7, Loss: 0.05472356826066971
Epoch: 8, Loss: 0.029909977689385414
Epoch: 9, Loss: 0.009236966259777546
Epoch: 10, Loss: 0.05081788823008537
Epoch: 11, Loss: 0.009971292689442635
Epoch: 12, Loss: 0.04613920673727989
Epoch: 13, Loss: 0.04811929911375046
Epoch: 14, Loss: 0.040765371173620224
Epoch: 15, Loss: 0.024608640000224113
Epoch: 16, Loss: 0.04773058742284775
Epoch: 17, Loss: 0.09104682505130768
Epoch: 18, Loss: 0.2619621157646179
Epoch: 19, Loss: 0.007320807781070471
Epoch: 20, Loss: 0.0905398279428482
Epoch: 21, Loss: 0.05753006413578987
Epoch: 22, Loss: 0.0626545175909996
Epoch: 23, Loss: 0.2529808580875397
Epoch: 24, Loss: 0.04659084603190422
Epoch: 25, Loss: 0.1826079785823822


In [18]:
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9922
